# 🚢 Predicting Titanic Survival: A Beginner's Walkthrough

## 🧭 Project Goals
- Understand the dataset
- Explore the data
- Clean the data
- Engineer features
- Build and evaluate models
- Prepare a Kaggle submission

In [30]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
# Check for missing data
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex            891
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
FamilySize       0
Embarked_Q       0
Embarked_S       0
dtype: int64

In [27]:
# Fill missing values before encoding
if 'Age' in train.columns:
    train['Age'] = train['Age'].fillna(train['Age'].median())
if 'Embarked' in train.columns:
    train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

In [26]:
# Feature engineering
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
if 'Embarked' in train.columns:
    train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define features and target
features = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked_Q', 'Embarked_S']
X = train[features].copy()
y = train['Survived']

# Fill any NaNs just in case (especially Age or Fare)
X = X.fillna(0)

# Split and train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Predict and evaluate
preds = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, preds))


Validation Accuracy: 0.8044692737430168


In [24]:
# Prepare test data and submission
# Map Sex and fill missing values
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].fillna(0)

# Feature engineering
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test['Age'] = test['Age'].fillna(train['Age'].median())
test['Fare'] = test['Fare'].fillna(train['Fare'].median())

# Encode Embarked if it exists
if 'Embarked' in test.columns:
    test = pd.get_dummies(test, columns=['Embarked'], drop_first=True)

# Ensure same columns as train
for col in ['Embarked_Q', 'Embarked_S']:
    if col not in test.columns:
        test[col] = 0

# Final NaN check for all features
for feature in features:
    if test[feature].isnull().sum() > 0:
        print(f"Filling missing values in {feature}")
        test[feature] = test[feature].fillna(0)

# Predict and export
predictions = model.predict(test[features])
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': predictions})
submission.to_csv('submission.csv', index=False)